In [1]:
import os
import pandas as pd
from fastbook import *
from fastai.tabular.all import *
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from dtreeviz.trees import *
import sklearn

def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)

traindf = pd.read_csv('/mnt/c/Users/jsult/Desktop/spacetitanic3/train.csv',low_memory=False)
testdf = pd.read_csv('/mnt/c/Users/jsult/Desktop/spacetitanic3/test.csv',low_memory= False)
traindf.dropna(inplace = True)

/home/jakeunix/miniforge3/envs/sptitanic/lib/python3.11/site-packages/fastbook/__init__.py:11: UserWarning: Missing `ipywidgets` - please install it
  except ModuleNotFoundError: warn("Missing `ipywidgets` - please install it")
/home/jakeunix/miniforge3/envs/sptitanic/lib/python3.11/site-packages/fastbook/__init__.py:13: UserWarning: Missing `sentencepiece` - please run `pip install 'sentencepiece<0.1.90'`
  except ModuleNotFoundError: warn("Missing `sentencepiece` - please run `pip install 'sentencepiece<0.1.90'`")


In [2]:
nas = list(testdf.isna().sum(axis = 1))
indexes = list(testdf.index)
keepinds = []
dropinds = []
for i in range(len(indexes)):
    if nas[i] == 0:
        keepinds.append(i)
    else:
        dropinds.append(i)
droptest = testdf.loc[dropinds].copy()
testdf = testdf.loc[keepinds].copy()

In [3]:
len(testdf)

3281

In [ ]:
splits = RandomSplitter(seed = 1)(range_of(traindf))
dep_var = 'Transported'
procs = [Categorify,FillMissing,Normalize]
cont,cat = cont_cat_split(traindf, 1, dep_var=dep_var)
to = TabularPandas(traindf, procs, cat, cont, y_names=dep_var,splits = splits)

xs,y = to.train.xs,to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

m = DecisionTreeRegressor(max_leaf_nodes=20)
m.fit(xs, y)

fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(m,
                   feature_names= traindf.columns,
                   filled=True)

print('decision tree prediction',((m.predict(valid_xs) > 0.5) == valid_y).sum() / len(valid_y))




In [ ]:
m = RandomForestClassifier(n_estimators= 100, max_leaf_nodes= 50)
m.fit(xs, y)
preds = m.predict(valid_xs)
print("mae",mean_absolute_error(valid_y, preds))
print('preds', (preds == valid_y).sum() /len(valid_y))


In [ ]:
cont,cat = cont_cat_split(testdf, 1, dep_var=dep_var)
mode_values = testdf[cat].mode().iloc[0]
testdf[cat] = testdf[cat].fillna(mode_values)


for column in cont:
    mode_value = testdf[column].median()  # Calculate the mode for the column
    testdf[column].fillna(mode_value, inplace=True)  # Fill missing values with mode
    
to_new = to.train.new(testdf)
to_new.process()

print(m.predict(to_new.xs))

testtran = [True if x == 1 else False for x in list(m.predict(to_new.xs))]
testdf['Transported'] = testtran


In [5]:
droptest['Transported'] = False


In [21]:
result = pd.concat([droptest,res])
result = result.sort_index()


In [22]:
ret = result[['PassengerId','Transported']]
ret

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4258,9238_05,False
4267,9260_01,False
4273,9269_01,False
4274,9271_01,False


In [29]:
testdf2 = pd.read_csv('/mnt/c/users/jsult/desktop/spacetitanic3/final/testpartial2.csv',low_memory= False)
res2 = testdf2[['PassengerId','Transported']]
result2 = pd.concat([droptest,res2])
result2 = result2.sort_index()
ret2 = result2[['PassengerId','Transported']]


In [31]:
(ret2 == ret1).sum()

PassengerId    4277
Transported    4236
dtype: int64

In [32]:
ret2.to_csv('submit2.csv',index=False)

In [25]:
testdf1 = pd.read_csv('/mnt/c/users/jsult/desktop/spacetitanic3/final/testpartial1.csv',low_memory= False)
res1 = testdf1[['PassengerId','Transported']]
result1 = pd.concat([droptest,res1])
result1 = result1.sort_index()
ret1 = result1[['PassengerId','Transported']]


In [27]:
(ret1 == ret).sum()

PassengerId    4277
Transported    4206
dtype: int64

In [19]:
testdf = pd.read_csv('/mnt/c/users/jsult/desktop/spacetitanic3/final/testpartial.csv',low_memory= False)


In [20]:
res = testdf[['PassengerId','Transported']]
res

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
3276,9263_01,True
3277,9265_01,True
3278,9266_01,True
3279,9266_02,True
